In [ ]:
import numpy as np
import gpflow
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow.utilities import print_summary, set_trainable, to_default_float
import statistics
from scipy import stats
import random
from gpflow.utilities import print_summary, positive
import math
from gpflow.ci_utils import ci_niter
#import plot_from_samples, colors
colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]
import scipy
from scipy import io

In [ ]:
def plot_from_samples(m, X, Y, parameters, parameter_samples, thin):

    f = plt.figure(figsize=(12, 6))
    a1 = f.add_axes([0.05, 0.05, 0.9, 0.6])
    a2 = f.add_axes([0.05, 0.7, 0.9, 0.1])
    a3 = f.add_axes([0.05, 0.85, 0.9, 0.1])

    xx = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)

    Fpred, Ypred = [], []
    num_samples = len(parameter_samples[0])
    for i in range(0, num_samples, thin):
        for parameter, var_samples in zip(parameters, parameter_samples):
            parameter.assign(var_samples[i])
        Ypred.append(m.predict_y(xx)[0])
        Fpred.append(np.squeeze(m.predict_f_samples(xx, 1)))

    for i in range(m.likelihood.num_classes):
        x = X[Y.reshape(-1) == i]
        (points,) = a3.plot(x, x * 0, ".")
        color = points.get_color()
        for F in Fpred:
            a1.plot(xx, F[:, i], color=color, lw=0.2, alpha=1.0)
        for Yp in Ypred:
            a2.plot(xx, Yp[:, i], color=color, lw=0.5, alpha=1.0)

    a2.set_ylim(-0.1, 1.1)
    a2.set_yticks([0, 1])
    a2.set_xticks([])

    a3.set_xticks([])
    a3.set_yticks([])

    a3.set_title("inputs X")
    a2.set_title(
        "predicted mean label value \
                 $\mathbb{E}_{q(\mathbf{u})}[y^*|x^*, Z, \mathbf{u}]$"
    )
    a1.set_title(
        "posterior process samples \
                $\int d\mathbf{u} q(\mathbf{u})p(f^*|\mathbf{u}, Z, x^*)$"
    )

In [ ]:
gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_jitter(1e-4)
gpflow.config.set_default_summary_fmt("notebook")
# convert to float64 for tfp to play nicely with gpflow in 64
f64 = gpflow.utilities.to_default_float

tf.random.set_seed(123)


%matplotlib inline

# 48 hr data, 1hr sampling, 1 repeat
We create synthetic data that should have a circadian rhythm:

In [ ]:
# toy data
N = 48
X = np.linspace(0,N-1,N).reshape((N,1))
Y = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3 +0.5

In [ ]:
# z-score it
Z = stats.zscore(Y)

To check the Z-score:

In [ ]:
def mean(data):
    n=len(data)
    mean=sum(data)/n
    return mean

In [ ]:
def variance(data):
    n=len(data)
    mean=sum(data)/n
    deviations=[(x-mean)**2 for x in data]
    variance=sum(deviations)/n
    return variance

In [ ]:
variance(Z)

In [ ]:
mean(Z)

In [ ]:
plt.plot(X,Y, label='Original toy data')
plt.plot(X,Z, label='Z-scored data')
plt.legend()
plt.xlabel("Time (hr)")
plt.title("48hr data, 1hr sampling")

Here we can see the circadian nature of the data.

## Fitting using a Squared Exponential Periodic Kernel

Here we use a Squared Exponential periodic kernel with period set to to be 24 hours.

The Squared Exponential base kernel in a periodic kernel can be expressed as:
\begin{equation}
k(r) = \sigma ^2 \exp\left(-\frac{1}{2}\sin ^2\left(\frac{\pi r/ \gamma}{\lambda ^2}\right)\right) ,
\end{equation}
where $r$ is the Euclidean distance between the input points, $\lambda$ is the lengthscales parameter, $\sigma^2$ is the variance parameter and $\gamma$ is the period parameter.

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5,55)

## Optimising using MLE

We will attempt to improve this fit using Maximum Likelihood Estimation.

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

Our fit still gives a period close to circadian rhythm.

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("Original 48 hr data 1hr Sampling MLE")

## Maximum A Priori Estimation using Priors

### Using priors with Gamma distribution

A random quantity $X$ has a Gamma distribution with parameters $\alpha \geq 1$ and $\beta$, written $X | \alpha, \beta ~$ Gamma($\alpha, \beta$), if it has pdf
\begin{equation}
f(x | \alpha,\beta) = 
\begin{cases} 
  \frac{\beta^{\alpha}}{\Gamma(\alpha)}x^{\alpha - 1}\exp({-\beta x}) & x\in [0,\infty]\\
  0 & otherwise \\
\end{cases}
\end{equation}
This is useful when modelling strictly positive quantities, as we want our prior to be positive, we will try using a Gamma prior. To choose the right prior values, we can see what each distribution looks like below.

In [ ]:
from scipy.stats import gamma
fig, ax = plt.subplots(1,2, figsize=(12, 4), constrained_layout=True)
a = 2
b = 1
N = 48
x = np.linspace(0,N-1,N)
ax[0].plot(x, gamma.pdf(x, a, b),label='$\Gamma(2,1)$')
ax[0].legend(loc='best', frameon=False)
ax[0].set_title("Gamma Prior: $\Gamma(2,1)$")
a = 25
ax[1].plot(x, gamma.pdf(x, a, b),label='$\Gamma(25,1)$')
ax[1].legend(loc='best', frameon=False)
ax[1].set_title("Gamma Prior: $\Gamma(25,1)$")
plt.show()

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 1hr Sampling MAP")

# 24hrs data, 1hr sampling, 1 repeat
### MLE

In [ ]:
# toy data
N = 24
X = np.linspace(0,N-1,N).reshape((N,1))
Y = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3 +0.5

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 1hr Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 1hr Sampling MAP")

# 48hrs data, 4hr sampling, 1 repeat
### MLE

In [ ]:
# toy data
N = 48
M = int(N/4) # to get more realistic sampling
X = np.linspace(0,N-1,M).reshape((M,1))
Y = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3 +0.5

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Y, label='Original toy data')
plt.plot(X,Z, label='Z-scored data')
plt.xlabel("Time (hr)")
plt.legend()
plt.title("48hrs 4hr sampling")

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 4hr Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 4hr Sampling MAP")

# 24hrs data, 4hr sampling, 1 repeat
### MLE

In [ ]:
# toy data
N = 24
M = int(N/4) # to get more realistic sampling
X = np.linspace(0,N-1,M).reshape((M,1))
Y = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3 +0.5

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Y, label='Original toy data')
plt.plot(X,Z, label='Z-scored data')
plt.xlabel("Time (hr)")
plt.title("24hr data 4hr sampling")
plt.legend()

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 4hr Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 4hr Sampling MAP")

# 48hrs data, 1hr sampling, 3 repeat
### MLE

In [ ]:
# toy data
N = 48
X = np.linspace(0,N-1,N).reshape((N,1))
Y1 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y2 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y3 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5

In [ ]:
X = np.concatenate([X, X, X])
Y = np.concatenate([Y1,Y2,Y3])

In [ ]:
plt.plot(X,Y,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Original 48 hr data 1hr sampling, 3 repeats")

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Z,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Z-scored 48 hr data 1hr sampling, 3 repeats")

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 1hr Sampling 3 repeats MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 1hr 3 repeats Sampling MAP")

# 24hrs data, 1hr sampling, 3 repeats
### MLE

In [ ]:
# toy data
N = 24
X = np.linspace(0,N-1,N).reshape((N,1))
Y1 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y2 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y3 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5

In [ ]:
X = np.concatenate([X, X, X])
Y = np.concatenate([Y1,Y2,Y3])

In [ ]:
plt.plot(X,Y,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Original 24 hr data 1hr sampling, 3 repeats")

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Z,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Z-scored 24 hr data 1hr sampling, 3 repeats")

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 1hr 3 repeats Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 1hr 3 repeats Sampling MAP")

# 48hrs data, 4hr sampling, 3 repeats
### MLE

In [ ]:
# toy data
N = 48
M = int(N/4)
X = np.linspace(0,N-1,M).reshape((M,1))
Y1 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5
Y2 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5
Y3 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5

In [ ]:
X = np.concatenate([X, X, X])
Y = np.concatenate([Y1,Y2,Y3])

In [ ]:
plt.plot(X,Y,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Original 48 hr data 4hr sampling, 3 repeats")

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Z,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Z-scored 48 hr data 4hr sampling, 3 repeats")

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 4hr 3 repeats Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("48 hr data 4hr 3 repeats Sampling MAP")

# 24hrs data, 4hr sampling, 3 repeats
### MLE

In [ ]:
# toy data
N = 24
M = int(N/4)
X = np.linspace(0,N-1,M).reshape((M,1))
Y1 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5
Y2 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5
Y3 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(M,1)*0.3+0.5

In [ ]:
X = np.concatenate([X, X, X])
Y = np.concatenate([Y1,Y2,Y3])

In [ ]:
plt.plot(X,Y,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Original 24 hr data 4hr sampling, 3 repeats")

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
plt.plot(X,Z,'o', color='black')
plt.xlabel("Time (hr)")
plt.title("Z-scored 24 hr data 4hr sampling, 3 repeats")

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.assign(0.1)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24 hr data 4hr 3 repeats Sampling MLE")

### MAP

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)

In [ ]:
m

In [ ]:
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_y(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X, Z, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
_ = plt.xlim(-5, 55)
plt.xlabel("Time (hr)")
plt.title("24hr data 4hr 3 repeats Sampling MAP")

# MCMC (Markov Chain Monte Carlo) with 48 hr, 1hr sampling, 3 repeats

In [ ]:
# toy data
N = 48
X = np.linspace(0,N-1,N).reshape((N,1))
Y1 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y2 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5
Y3 = np.sin(np.pi*X/12) + np.cos(np.pi*X/12) + np.random.randn(N,1)*0.3+0.5

In [ ]:
X = np.concatenate([X, X, X])
Y = np.concatenate([Y1,Y2,Y3])

In [ ]:
# z-score it
Z = stats.zscore(Y)

In [ ]:
# create kernel
k=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)

In [ ]:
k.base_kernel.variance.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.base_kernel.lengthscales.prior = tfp.distributions.Gamma(
    to_default_float(2), to_default_float(1)
)
k.period.prior = tfp.distributions.Gamma(
   to_default_float(25), to_default_float(1)
)
k

In [ ]:
# create model using data and above kernel
m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
m.likelihood.variance.prior = tfp.distributions.Gamma(
   to_default_float(2), to_default_float(0.1)
)
m

In [ ]:
opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)
m

In [ ]:
data=(X,Z)
model = m
num_burnin_steps = ci_niter(1000)
num_samples = ci_niter(5000)

# Note that here we need model.trainable_parameters, not trainable_variables - only parameters can have priors!
hmc_helper = gpflow.optimizers.SamplingHelper(
    model.log_posterior_density, model.trainable_parameters
)

hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=hmc_helper.target_log_prob_fn, num_leapfrog_steps=10, step_size=0.01
)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    hmc, num_adaptation_steps=10, target_accept_prob=f64(0.75), adaptation_rate=0.1
)


@tf.function
def run_chain_fn():
    return tfp.mcmc.sample_chain(
        num_results=num_samples,
        num_burnin_steps=num_burnin_steps,
        current_state=hmc_helper.current_state,
        kernel=adaptive_hmc,
        trace_fn=lambda _, pkr: pkr.inner_results.is_accepted,
    )


samples, traces = run_chain_fn()
parameter_samples = hmc_helper.convert_to_constrained_values(samples)

param_to_name = {param: name for name, param in gpflow.utilities.parameter_dict(model).items()}

In [ ]:
def plot_samples(samples, parameters, y_axis_label):
    plt.figure(figsize=(8, 4))
    for val, param in zip(samples, parameters):
        plt.plot(tf.squeeze(val), label=param_to_name[param])
    plt.legend(bbox_to_anchor=(1.0, 1.0))
    plt.xlabel("HMC iteration")
    plt.ylabel(y_axis_label)


plot_samples(samples, model.trainable_parameters, "unconstrained values")
plot_samples(parameter_samples, model.trainable_parameters, "constrained parameter values")

In [ ]:
def marginal_samples(samples, parameters, y_axis_label):
    fig, axes = plt.subplots(1, len(param_to_name), figsize=(15, 3), constrained_layout=True)
    for ax, val, param in zip(axes, samples, parameters):
        ax.hist(np.stack(val).flatten(), bins=20)
        ax.set_title(param_to_name[param])
    fig.suptitle(y_axis_label)
    plt.show()


marginal_samples(samples, model.trainable_parameters, "Unconstrained variable samples")
marginal_samples(parameter_samples, model.trainable_parameters, "Constrained parameter samples")

In [ ]:
# plot the function posterior
xx = np.linspace(-0.1, 48.1, 100)[:, None]
plt.figure(figsize=(12, 6))

for i in range(0, num_samples, 20):
    for var, var_samples in zip(hmc_helper.current_state, samples):
        var.assign(var_samples[i])
    f = model.predict_f_samples(xx, 1)
    plt.plot(xx, f[0, :, :], "C0", lw=2, alpha=0.3)

plt.plot(X, Z, "kx", mew=2)
_ = plt.xlim(xx.min(), xx.max())
_ = plt.ylim(-2, 2)
plt.xlabel("$x$")
plt.ylabel("$f|X,Z$")
plt.title("Posterior GP samples")

plt.show()